In [1]:
import cv2
from picamera2 import Picamera2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import pickle
import os
os.environ["DISPLAY"] = ":0"


In [2]:

model_path = "/home/pi/EZLift/src/raspberry_pi/256_edgetpu.tflite"

model = YOLO(model_path, task='detect')

In [3]:
# import time
# import cv2

# picam2 = Picamera2()
# picam2.preview_configuration.main.size = (3280, 2464)
# picam2.preview_configuration.main.format = "RGB888"
# picam2.preview_configuration.align()
# picam2.configure("preview")
# picam2.start()
# picam2.start(show_preview=False)
# frame = picam2.capture_array()

# with open("frame.pkl", "wb") as f:
#     pickle.dump(frame, f)

In [ ]:
import time
import cv2

picam2 = Picamera2()
picam2.preview_configuration.main.size = (3280, 2464)
picam2.preview_configuration.main.format = "RGB888"
picam2.preview_configuration.align()
picam2.configure("preview")
picam2.start()
picam2.start(show_preview=False)

while True:
    start_time = time.time()
    
    # Capture frame-by-frame
    frame = picam2.capture_array()

    # Run YOLO inference on the frame with imgsz=192
    # results = model.predict(frame, device="tpu:0", imgsz=192)
    frame = cv2.resize(frame, (256, 256))
    results = model.predict(frame, device="tpu:0", imgsz=256)


    # Visualize the results on the frame
    # annotated_frame = results[0].plot()

    # Calculate FPS
    end_time = time.time()
    fps = 1.0 / (end_time - start_time)
    
    # Overlay the FPS on the annotated frame
    # cv2.putText(annotated_frame, f"FPS: {fps:.2f}", (10, 30), 
    #             cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display the resulting frame
    # cv2.imshow("Camera", annotated_frame)

    # Print FPS to the console (optional)
    print(f"FPS: {fps:.2f}")

    # # Break the loop if 'q' is pressed
    # if cv2.waitKey(1) == ord("q"):
    #     break

# cv2.destroyAllWindows()

[0:14:37.494135327] [3562]  INFO Camera camera_manager.cpp:327 libcamera v0.4.0+53-29156679
[0:14:37.546948171] [3634]  WARN RPiSdn sdn.cpp:40 Using legacy SDN tuning - please consider moving SDN inside rpi.denoise
[0:14:37.549013707] [3634]  WARN RPI vc4.cpp:393 Mismatch between Unicam and CamHelper for embedded data usage!
[0:14:37.549700830] [3634]  INFO RPI vc4.cpp:447 Registered camera /base/soc/i2c0mux/i2c@1/imx219@10 to Unicam device /dev/media4 and ISP device /dev/media0
[0:14:37.549744089] [3634]  INFO RPI pipeline_base.cpp:1121 Using configuration file '/usr/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'
[0:14:37.562657483] [3562]  INFO Camera camera.cpp:1202 configuring streams: (0) 3264x2464-RGB888 (1) 3280x2464-SBGGR10_CSI2P
[0:14:37.563105608] [3634]  INFO RPI vc4.cpp:622 Sensor: /base/soc/i2c0mux/i2c@1/imx219@10 - Selected sensor format: 3280x2464-SBGGR10_1X10 - Selected unicam format: 3280x2464-pBAA


Loading /home/pi/EZLift/src/raspberry_pi/256_edgetpu.tflite on device 0 for TensorFlow Lite Edge TPU inference...

0: 256x256 1 tie, 39.1ms
Speed: 8.5ms preprocess, 39.1ms inference, 5.1ms postprocess per image at shape (1, 3, 256, 256)
FPS: 0.16

0: 256x256 (no detections), 24.4ms
Speed: 4.7ms preprocess, 24.4ms inference, 2.4ms postprocess per image at shape (1, 3, 256, 256)
FPS: 9.98

0: 256x256 (no detections), 22.1ms
Speed: 2.3ms preprocess, 22.1ms inference, 2.6ms postprocess per image at shape (1, 3, 256, 256)
FPS: 14.55

0: 256x256 (no detections), 26.4ms
Speed: 3.2ms preprocess, 26.4ms inference, 2.6ms postprocess per image at shape (1, 3, 256, 256)
FPS: 12.73

0: 256x256 (no detections), 22.3ms
Speed: 2.5ms preprocess, 22.3ms inference, 2.7ms postprocess per image at shape (1, 3, 256, 256)
FPS: 15.96

0: 256x256 (no detections), 21.4ms
Speed: 3.1ms preprocess, 21.4ms inference, 2.7ms postprocess per image at shape (1, 3, 256, 256)
FPS: 13.58

0: 256x256 (no detections), 21.3m

In [ ]:
input_path = r"/home/pi/EZLift/src/raspberry_pi/notebooks/test_imgs/istockphoto-1318934972-612x612.jpg"
img = cv2.imread(input_path)
img = cv2.resize(img, (256, 256))
results = model.predict(img, device="tpu:0", imgsz=256)

0: 256x256 6 persons, 29.4ms
Speed: 5.8ms preprocess, 29.4ms inference, 12.9ms postprocess per image at shape (1, 3, 256, 256)


In [ ]:
boxes = results[0].boxes
x1, y1, x2, y2 = boxes[0].xyxy[0].cpu().numpy()
x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
conf = boxes[0].conf[0].cpu().numpy()
cls_id = int(boxes[0].cls[0].cpu().numpy())

bb_crop = results[0].orig_img[y1:y2, x1:x2]